In [158]:
#Download the libraries
import nltk
import re
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
import sys
import time
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import tree

Setting working directory

In [159]:
print os.getcwd();

/Users/imacair/Desktop/Products3


In [160]:
os.chdir("/Users/imacair/Desktop/Products3/")

Read the file

In [161]:
data= pd.read_csv('Final_Manual_3006.csv',
                    encoding='latin-1',delimiter=',')

In [162]:
#Converts text into ASCII

In [163]:
#data.text = data.text.str.encode('ascii','replace')

In [164]:
data.message = data.message.str.encode('ascii','replace')

In [165]:
len(data)

4685

In [166]:
#sample_data= data.sample(n=64000)

In [167]:
sample_data= data

In [168]:
#data_t=sample_data["text"]

In [169]:
data_t=sample_data["message"]

In [170]:
len(data_t)

4685

In [171]:
#lowercase
data_t = data_t.str.lower()


In [172]:
data_s=sample_data["sentiment"]

In [173]:
np.unique(data_s)

array([u'1', u'2', u'3', u'4', u'N', u'n'], dtype=object)

In [174]:
#data_s= data_s[~np.isnan(data_s)]

In [175]:
final = data
res5= pd.DataFrame( index=range(0,len(data_t)),columns = {'new_sent'} )
res5[(final.sentiment==u'2')] = '-1'
res5[(final.sentiment==u'1')] = '-1'
res5[(final['sentiment']==u'3')] = '1'
res5[(final['sentiment']==u'4')] = '1'
res5[(final['sentiment']==u'N')] = '0'
res5[(final['sentiment']==u"n")] = '0'
final=pd.concat([final, res5], axis=2)

In [176]:
#final['sentiment'] = final['sentiment'][~pd.isnull(final['sentiment'])]

In [177]:
#final['sentiment']=np.nan_to_num(final['sentiment'])
#final['sentiment'] = final[~np.isnan('sentiment')]
#y = 
#final['sentiment'][~np.isfinite(final['sentiment'])]

In [178]:
np.unique(final.new_sent)

array(['-1', '0', '1'], dtype=object)

Abbriviation translation

In [179]:
with open('abbrev.csv', mode='r') as infile:
    reader = csv.reader(infile)
    replacement = {rows[0].lower():rows[1].lower() for rows in reader              
                  }

In [180]:
#replacement

In [181]:
#replacement = {
##'r':'are',
#'y':'why',
#'u':'you'}

In [182]:
##How in works
s1 = 'y r u l8'

s2 = ' '.join([replacement.get(w, w) for w in s1.split()])
s2

'why? -or- yes are you l8'

In [183]:
result = pd.DataFrame()
result = final

In [184]:
for i in range(len(result)):
    data_t.values[i]=' '.join([replacement.get(w, w) for w in data_t.values[i].split()])

In [185]:

text = data_t.to_string()
text = nltk.word_tokenize(text)
fdist = nltk.FreqDist(text)
s2 = set([w for w in set(text) if len(w) > 2 and fdist[w] > 2])
for i in range(len(result)):
    data_t.values[i]=' '.join(filter(lambda w: w in s2,data_t.values[i].split()))

In [186]:
from nltk.corpus import stopwords
s=set(stopwords.words('english'))
for i in range(len(result)):
    data_t.values[i]=' '.join(filter(lambda w: not w in s,data_t.values[i].split()))

In [187]:
data_t

0       medical south ankylosing x-ray imaging south m...
1       real time -or- retweet also works national ank...
2                                  ankylosing spondylitis
3       real time -or- retweet blog post bacteria invo...
4       ankylosing spondylitis symptoms finally diagnosed
5       always ante support great work diploma ophthal...
6                 inflammatory back pain group ankylosing
7             today world ankylosing spondylitis day ante
8       ante person world health organization inflamma...
9                                  ankylosing spondylitis
10                                 ankylosing spondylitis
11      real time -or- retweet national ankylosing spo...
12                             latest blog post interview
13      liked ante video like live ankylosing spondylitis
14      real time -or- retweet ankylosing spondylitis ...
15                                             ankylosing
16      real time -or- retweet also works national ank...
17      perman

In [188]:
data_t.head(10)

0    medical south ankylosing x-ray imaging south m...
1    real time -or- retweet also works national ank...
2                               ankylosing spondylitis
3    real time -or- retweet blog post bacteria invo...
4    ankylosing spondylitis symptoms finally diagnosed
5    always ante support great work diploma ophthal...
6              inflammatory back pain group ankylosing
7          today world ankylosing spondylitis day ante
8    ante person world health organization inflamma...
9                               ankylosing spondylitis
Name: message, dtype: object

In [189]:
#lowercase
data_t = data_t.str.lower()
#Remove urls
data_t= data_t.str.replace(r'(http.*) |(http.*)$|\n', "",)
#Remove twitter handles
data_t = data_t.str.replace(r"@\\w+", "")
#remove htmls
data_t = data_t.str.replace(r'<.*?>', "")
#Remove citations
data_t = data_t.str.replace(r'@[a-zA-Z0-9]*', "")

#remove _
#data_t = data_t.str.replace(r'\_+',"")

In [190]:
data_t.head(10)

0    medical south ankylosing x-ray imaging south m...
1    real time -or- retweet also works national ank...
2                               ankylosing spondylitis
3    real time -or- retweet blog post bacteria invo...
4    ankylosing spondylitis symptoms finally diagnosed
5    always ante support great work diploma ophthal...
6              inflammatory back pain group ankylosing
7          today world ankylosing spondylitis day ante
8    ante person world health organization inflamma...
9                               ankylosing spondylitis
Name: message, dtype: object

Spliting the data


In [191]:
from sklearn.cross_validation import train_test_split
import numpy as np
from sklearn.model_selection import KFold

In [192]:
data_train, data_test, label_train, label_test = train_test_split(data_t, final.new_sent, test_size=0.3, random_state=2340)

In [193]:
#data_train, data_test, label_train, label_test = train_test_split(data_t, data_s, test_size=0.3, random_state=2340)

In [194]:
t0 = time.time()
vectorizer = TfidfVectorizer(    sublinear_tf=True,
                                 use_idf=True,stop_words = 'english')
train_vectors = vectorizer.fit_transform(data_train)
test_vectors = vectorizer.transform(data_test)
t1 = time.time()
time_vec = t1-t0

In [195]:
print(time_vec)

0.154000043869


In [196]:
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(C=0.6, kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, label_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

In [197]:
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(C=0.6, kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, label_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

In [198]:
print("Results for SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
print(classification_report(label_test, prediction_linear))
confusion_matrix(label_test, prediction_linear)

Results for SVC(kernel=linear)
Training time: 0.882836s; Prediction time: 0.218120s
             precision    recall  f1-score   support

         -1       0.66      0.61      0.63       391
          0       0.64      0.78      0.70       576
          1       0.72      0.56      0.63       439

avg / total       0.67      0.66      0.66      1406



array([[238, 117,  36],
       [ 69, 448,  59],
       [ 55, 140, 244]])

In [199]:
# Perform classification with SVM, kernel=linear
classifier_liblinear = svm.LinearSVC()
t0 = time.time()
classifier_liblinear.fit(train_vectors, label_train)
t1 = time.time()
prediction_liblinear = classifier_liblinear.predict(test_vectors)
t2 = time.time()
time_liblinear_train = t1-t0
time_liblinear_predict = t2-t1

In [200]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [201]:
# Perform classification with random forest
classifier_rf = RandomForestClassifier()
t0 = time.time()
classifier_rf.fit(train_vectors, label_train)
t1 = time.time()
prediction_rf = classifier_rf.predict(test_vectors)
t2 = time.time()
time_rf_train = t1-t0
time_rf_predict = t2-t1

In [202]:
# Perform classification with Multinomial Naïve Bayes.
classifier_nb = MultinomialNB()
t0 = time.time()
classifier_nb.fit(train_vectors, label_train)
t1 = time.time()
prediction_nb = classifier_nb.predict(test_vectors)
t2 = time.time()
time_nb_train = t1-t0
time_nb_predict = t2-t1

In [203]:

print("Results for SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
print(classification_report(label_test, prediction_linear))
print("Results for LinearSVC()")
print("Training time: %fs; Prediction time: %fs" % (time_liblinear_train, time_liblinear_predict))
print(classification_report(label_test, prediction_liblinear))
print("Results for MultinomialNB()")
print("Training time: %fs; Prediction time: %fs" % (time_nb_train, time_nb_predict))
print(classification_report(label_test, prediction_nb))

Results for SVC(kernel=linear)
Training time: 0.882836s; Prediction time: 0.218120s
             precision    recall  f1-score   support

         -1       0.66      0.61      0.63       391
          0       0.64      0.78      0.70       576
          1       0.72      0.56      0.63       439

avg / total       0.67      0.66      0.66      1406

Results for LinearSVC()
Training time: 0.041947s; Prediction time: 0.000941s
             precision    recall  f1-score   support

         -1       0.63      0.58      0.61       391
          0       0.63      0.71      0.67       576
          1       0.64      0.58      0.61       439

avg / total       0.64      0.64      0.63      1406

Results for MultinomialNB()
Training time: 0.017263s; Prediction time: 0.000889s
             precision    recall  f1-score   support

         -1       0.64      0.58      0.61       391
          0       0.62      0.76      0.68       576
          1       0.67      0.53      0.59       439

avg / to

Here we will try to ensemble

In [204]:
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10, random_state=8)

In [205]:
estimators = []
#estimators.append(('rbf_svm', classifier_rbf ))
estimators.append(('linear_svm',classifier_liblinear))
estimators.append(('linear2_svm',classifier_linear))
estimators.append(('random_forest',classifier_rf))
estimators.append(('multi_naive',classifier_nb))
#estimators.append(('lstm',model))

ensemble = VotingClassifier(estimators)

In [206]:

results = model_selection.cross_val_score(ensemble, train_vectors, label_train, cv=kfold)
print(results.mean())

0.642583165511


In [207]:
label_tests = np.asarray(label_test)

In [208]:
label_tests

array(['0', '0', '0', ..., '0', '1', '-1'], dtype=object)

In [209]:
df =[]
df.append('linear_svm')
df.append('linear2_svm')
df.append('random_forest')
df.append('multi_naive')
df.append('label_tests')

In [210]:
prediction_linear

array(['0', '-1', '0', ..., '1', '0', '-1'], dtype=object)

In [211]:
df

['linear_svm', 'linear2_svm', 'random_forest', 'multi_naive', 'label_tests']

In [212]:
df=[prediction_linear, prediction_liblinear, prediction_nb,label_tests]

In [213]:
df = pd.DataFrame(df)
df = df.transpose()
df.columns = ['prediction_linear', 'prediction_liblinear', 'prediction_nb','label_tests']
#df

In [214]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier
import numpy as np


lr = RandomForestClassifier()
sclf = StackingClassifier(classifiers=[classifier_liblinear,classifier_linear,classifier_nb], 
                          meta_classifier=lr)

print('3-fold cross validation:\n')

for clf, label in zip([classifier_liblinear,classifier_linear, classifier_nb,sclf], 
                      ['linear_svm', 
                       'linear2_svm',
                       'multi_naive',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, train_vectors, label_train, 
                                              cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

3-fold cross validation:

Accuracy: 0.62 (+/- 0.01) [linear_svm]
Accuracy: 0.62 (+/- 0.01) [linear2_svm]
Accuracy: 0.61 (+/- 0.02) [multi_naive]
Accuracy: 0.62 (+/- 0.01) [StackingClassifier]


In [215]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier
import numpy as np


lr = RandomForestClassifier()
sclf = StackingClassifier(classifiers=[classifier_liblinear,classifier_linear, classifier_nb], 
                          meta_classifier=lr)

print('3-fold cross validation:\n')

for clf, label in zip([classifier_liblinear,classifier_linear,classifier_nb,sclf], 
                      ['linear_svm', 
                       'linear2_svm',
                       'multi_naive',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, train_vectors, label_train, 
                                              cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

3-fold cross validation:

Accuracy: 0.64 (+/- 0.03) [linear_svm]
Accuracy: 0.64 (+/- 0.03) [linear2_svm]
Accuracy: 0.63 (+/- 0.04) [multi_naive]
Accuracy: 0.64 (+/- 0.03) [StackingClassifier]


In [216]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier
import numpy as np


lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[classifier_liblinear,classifier_linear, classifier_nb], 
                          meta_classifier=lr)

print('3-fold cross validation:\n')

for clf, label in zip([classifier_liblinear,classifier_linear,classifier_nb,sclf], 
                      ['linear_svm', 
                       'linear2_svm',
                       'multi_naive',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, train_vectors, label_train, 
                                              cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

3-fold cross validation:

Accuracy: 0.64 (+/- 0.03) [linear_svm]
Accuracy: 0.64 (+/- 0.03) [linear2_svm]
Accuracy: 0.63 (+/- 0.04) [multi_naive]
Accuracy: 0.64 (+/- 0.03) [StackingClassifier]


In [151]:
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier
import numpy as np


lr = classifier_linear
sclf = StackingClassifier(classifiers=[classifier_liblinear, classifier_nb], 
                          meta_classifier=lr)

print('3-fold cross validation:\n')

for clf, label in zip([classifier_liblinear,classifier_nb,sclf], 
                      ['linear_svm', 
                       'multi_naive',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, train_vectors, label_train, 
                                              cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

3-fold cross validation:

Accuracy: 0.62 (+/- 0.03) [linear_svm]
Accuracy: 0.62 (+/- 0.03) [multi_naive]
Accuracy: 0.62 (+/- 0.03) [StackingClassifier]


In [152]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier
import numpy as np


lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[classifier_liblinear,classifier_linear, classifier_nb], 
                          meta_classifier=lr)


sclf.fit(train_vectors, label_train)

prediction_sclf = sclf.predict(test_vectors)
confusion_matrix(label_test, prediction_sclf)

array([[233, 115,  43],
       [ 84, 419,  73],
       [ 58, 130, 251]])

In [153]:

prediction_sclf = sclf.predict(test_vectors)

In [154]:
#Convert to np arrays
label_tests = np.asarray(label_test)


#Create a Data Frame
df=[ prediction_linear, prediction_liblinear,prediction_nb,prediction_sclf]
df = pd.DataFrame(df)
df = df.transpose()
df.columns = ['prediction_linear', 'prediction_liblinear','prediction_nb','staking']
df

,prediction_linear,prediction_liblinear,prediction_nb,staking
0,0,0,0,0
1,-1,-1,-1,-1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,-1,-1,-1,-1
9,1,1,0,1


In [155]:
#Convert to np arrays
label_tests = np.asarray(label_test)


#Create a Data Frame
df=[ prediction_linear, prediction_liblinear,prediction_nb]
df = pd.DataFrame(df)
df = df.transpose()
df.columns = ['prediction_linear', 'prediction_liblinear','prediction_nb']
df

,prediction_linear,prediction_liblinear,prediction_nb
0,0,0,0
1,-1,-1,-1
2,0,0,0
3,0,0,0
4,0,0,0
5,0,0,0
6,0,0,0
7,0,0,0
8,-1,-1,-1
9,1,1,0


In [156]:
from collections import Counter
df2= pd.DataFrame( index=range(0,len(data_test)),columns = {'final'} )
for i in range(len(data_test)):
    d=Counter(df.ix[i,:])
    dfin=d.most_common(1)[0][0]
    df2.values[i]= dfin
df["final"] = df2

In [157]:
df

,prediction_linear,prediction_liblinear,prediction_nb,final
0,0,0,0,0
1,-1,-1,-1,-1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,-1,-1,-1,-1
9,1,1,0,1
